In [1]:

from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os
from os import listdir
from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt
#from __future__ import print_function

from transform import *
#estas son mis funciones traidas del jupyter contiguo

In [2]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [3]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

In [ ]:
# Division entre la x y la y para test
#Ojo, normalmente convertiriamos el onehotencoder antes de esta division para ahorrarnos un paso

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [ ]:
#red neuronal propiamente dicha

In [6]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
# proceso de cambio de las y para poder utilizarlas en el modelo. 
# las necesitamos en el formato de array sin strings y getdummies no nos sirvio ya que 
# no podiamos obtener el color con posterioridad.

In [8]:
# importo aqui para que quede reflejado. 
from sklearn.preprocessing import OneHotEncoder

In [9]:
y_test


array(['white', 'black', 'white', ..., 'white', 'black', 'white'],
      dtype='<U5')

In [10]:
# Hacemos el reshape, luego onehot encoder, luego fit y por ultimo toarray
y_test= y_test.reshape(-1, 1)
y_test

array([['white'],
       ['black'],
       ['white'],
       ...,
       ['white'],
       ['black'],
       ['white']], dtype='<U5')

In [11]:
##3
enc = OneHotEncoder()
enc.fit(y_test)  

OneHotEncoder()

In [12]:
y_test = enc.transform(y_test).toarray()

In [13]:
#por fin y_test esta en la forma que queremos
y_test


array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [ ]:
#ahora hago lo mismo con y Y_train

In [14]:
y_train

array(['white', 'white', 'black', ..., 'black', 'white', 'white'],
      dtype='<U5')

In [15]:
y_train= y_train.reshape(-1, 1)

In [16]:
enc = OneHotEncoder()
enc.fit(y_train)  

OneHotEncoder()

In [17]:
y_train = enc.transform(y_train).toarray()

In [18]:
y_train

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [19]:
#ha funcionado. Tenemos y_test e y_train en el formato que necesitabamos
# podiamos haberlo hecho antes de dividir el split del train pero he decidido hacerlo dos veces para practicar

In [20]:
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [21]:
enc.inverse_transform([[0,1,0,0]])

array([['green']], dtype='<U5')

In [ ]:
# Codigo para para correr el modelo de red neuronal. 

In [22]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 12s 140ms/step - loss: 0.3361 - accuracy: 0.7252 - val_loss: 0.2005 - val_accuracy: 0.8460
Epoch 2/20
84/84 [==============================] - 13s 150ms/step - loss: 0.1998 - accuracy: 0.8579 - val_loss: 0.1492 - val_accuracy: 0.8856
Epoch 3/20
84/84 [==============================] - 14s 167ms/step - loss: 0.1508 - accuracy: 0.8943 - val_loss: 0.1037 - val_accuracy: 0.9213
Epoch 4/20
84/84 [==============================] - 14s 168ms/step - loss: 0.1041 - accuracy: 0.9245 - val_loss: 0.0759 - val_accuracy: 0.9528
Epoch 5/20
84/84 [==============================] - 14s 169ms/step - loss: 0.0818 - accuracy: 0.9440 - val_loss: 0.0705 - val_accuracy: 0.9561
Epoch 6/20
84/84 [==============================] - 14s 168ms/step - loss: 0.0717 - accuracy: 0.9529 - val_loss: 0.0622 - val_accuracy: 0.9619
Epoch 7/20
84/84 [==============================] - 14s 171ms/step - loss: 0.0616 - accuracy: 0.9620 - val_loss: 0.0544 - val_accuracy: 0.9628

In [23]:
X_test[45].shape
model.predict(np.array([X_test[403]]))

array([[8.6263273e-05, 2.8926894e-04, 4.2412998e-06, 9.9938536e-01]],
      dtype=float32)